In [1]:
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import AutoTokenizer
from datasets import load_dataset, Dataset, Features
from transformers import DataCollatorWithPadding
import evaluate
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_excel("Dataset.xlsx")
data = data.rename(columns={'Sentence': 'text', 'Label': 'label'})
data = data.sample(frac = 1).reset_index(drop=True)
data

,text,label
0,"Depending on the emotion, an appropriate measu...",Purpose
1,We apply this principle to gigapixel image ren...,Methods
2,"In this study, a random forest model with two ...",Methods
3,We present a framework for acuity-driven visua...,Purpose
4,"For this purpose, a procedure for characterizi...",Methods
...,...,...
246,"At last, we discuss several trends in auto-par...",Methods
247,"Using graphical demonstration, the impact of v...",Methods
248,It can not only enable the clients to preserve...,Background of Research
249,Results showed that the novel Hybrid RuDSTCs c...,Results


Test 1

In [3]:
sbert = SentenceTransformer('all-MiniLM-L12-v2')
X = sbert.encode(data['Sentence'])
y = data['Label'].replace({'a1': 0, 'a2': 1, 'a3' : 2, 'a4' : 3, 'a5' : 4}, inplace=False).to_numpy()
print(X.shape)
print(y.shape)

(251, 384)
(251,)


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 24)

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1024, activation = "relu"),
    tf.keras.layers.Dense(1024, activation = "relu"),
    tf.keras.layers.Dense(512, activation = "relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512, activation = "relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256, activation = "relu"),
    tf.keras.layers.Dense(128, activation = "relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation = "relu"),
    
    tf.keras.layers.Dense(32, activation = "relu"),
    tf.keras.layers.Dense(5, activation = "softmax")
])

In [26]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
history = model.fit(X_train, y_train, epochs = 20, batch_size = 16)

Epoch 1/20
11/11 [==============================] - 2s 32ms/step - loss: 0.1283 - accuracy: 0.9771
Epoch 2/20
11/11 [==============================] - 0s 32ms/step - loss: 0.5125 - accuracy: 0.9543
Epoch 3/20
11/11 [==============================] - 0s 33ms/step - loss: 0.1263 - accuracy: 0.9657
Epoch 4/20
11/11 [==============================] - 0s 34ms/step - loss: 0.1204 - accuracy: 0.9714
Epoch 5/20
11/11 [==============================] - 0s 35ms/step - loss: 0.1744 - accuracy: 0.9714
Epoch 6/20
11/11 [==============================] - 0s 36ms/step - loss: 0.0819 - accuracy: 0.9771
Epoch 7/20
11/11 [==============================] - 0s 35ms/step - loss: 0.0388 - accuracy: 0.9886
Epoch 8/20
11/11 [==============================] - 0s 34ms/step - loss: 0.0478 - accuracy: 0.9829
Epoch 9/20
11/11 [==============================] - 0s 34ms/step - loss: 0.0295 - accuracy: 0.9829
Epoch 10/20
11/11 [==============================] - 0s 34ms/step - loss: 0.1436 - accuracy: 0.9714
Epoch 11/

In [27]:
y_pred = model.predict(X_test)
y_pred_class = np.argmax(y_pred, axis = 1)
print(classification_report(y_pred_class, y_test))

3/3 [==============================] - 0s 5ms/step
              precision    recall  f1-score   support

           0       0.27      0.25      0.26        12
           1       0.40      0.40      0.40        10
           2       0.59      0.62      0.60        26
           3       0.46      0.43      0.44        14
           4       0.33      0.36      0.34        14

    accuracy                           0.45        76
   macro avg       0.41      0.41      0.41        76
weighted avg       0.44      0.45      0.45        76



Test 2

In [3]:

features = Features.from_dict({'text': {'dtype': 'string', 'id': None, '_type': 'Value'}, 'label': {'names': data['label'].unique().tolist(), 'id': None, '_type': 'ClassLabel'}})

train = data.groupby('label').head(10).reset_index(drop=True)
test = data.groupby('label').tail(-10).reset_index(drop=True)
train = Dataset.from_pandas(train, features=features)
test = Dataset.from_pandas(test, features=features)
print(train, test)

Dataset({
    features: ['text', 'label'],
    num_rows: 50
}) Dataset({
    features: ['text', 'label'],
    num_rows: 201
})


In [4]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

train = train.map(preprocess_function)
test = test.map(preprocess_function)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 201/201 [00:00<00:00, 909.85 examples/s]


In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

accuracy = evaluate.load("accuracy")
id2label = {i: label for i, label in enumerate(data['label'].unique().tolist())}
label2id = {label: i for i, label in enumerate(data['label'].unique().tolist())}

training_args = TrainingArguments(
    output_dir="/",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

In [11]:
model2 = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=5, id2label=id2label, label2id=label2id
)

trainer = Trainer(
    model=model2,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 10%|█         | 4/40 [00:45<02:31,  4.22s/it] 

{'eval_loss': 1.6099519729614258, 'eval_accuracy': 0.17412935323383086, 'eval_runtime': 23.3425, 'eval_samples_per_second': 8.611, 'eval_steps_per_second': 0.557, 'epoch': 1.0}


 20%|██        | 8/40 [01:34<03:58,  7.44s/it] 

{'eval_loss': 1.6069296598434448, 'eval_accuracy': 0.18407960199004975, 'eval_runtime': 22.9251, 'eval_samples_per_second': 8.768, 'eval_steps_per_second': 0.567, 'epoch': 2.0}


 30%|███       | 12/40 [02:23<03:38,  7.79s/it]

...

{'eval_loss': 1.5909351110458374, 'eval_accuracy': 0.19402985074626866, 'eval_runtime': 21.2672, 'eval_samples_per_second': 9.451, 'eval_steps_per_second': 0.611, 'epoch': 10.0}


100%|██████████| 40/40 [07:46<00:00, 11.66s/it]


{'train_runtime': 466.9598, 'train_samples_per_second': 1.071, 'train_steps_per_second': 0.086, 'train_loss': 1.5626529693603515, 'epoch': 10.0}


100%|██████████| 13/13 [00:20<00:00,  1.58s/it]


{'eval_loss': 1.5909351110458374,
 'eval_accuracy': 0.19402985074626866,
 'eval_runtime': 21.4278,
 'eval_samples_per_second': 9.38,
 'eval_steps_per_second': 0.607,
 'epoch': 10.0}

In [12]:
tokenizer2 = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
data_collator2 = DataCollatorWithPadding(tokenizer=tokenizer2)
model3 = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english", num_labels=5, id2label = id2label, label2id = label2id, ignore_mismatched_sizes=True

)

def preprocess_function(examples):
    return tokenizer2(examples["text"], truncation=True)

train1 = train.map(preprocess_function)
test1 = test.map(preprocess_function)

trainer2 = Trainer(
    model=model3,
    args=training_args,
    train_dataset=train1,
    eval_dataset=test1,
    tokenizer=tokenizer2,
    data_collator=data_collator2,
    compute_metrics=compute_metrics,
)

trainer2.train()

c:\Python311\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dhair\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are new

{'eval_loss': 1.6149466037750244, 'eval_accuracy': 0.17412935323383086, 'eval_runtime': 23.3535, 'eval_samples_per_second': 8.607, 'eval_steps_per_second': 0.557, 'epoch': 1.0}


 20%|██        | 8/40 [01:28<03:41,  6.92s/it] 

{'eval_loss': 1.6122069358825684, 'eval_accuracy': 0.17412935323383086, 'eval_runtime': 19.6944, 'eval_samples_per_second': 10.206, 'eval_steps_per_second': 0.66, 'epoch': 2.0}


 30%|███       | 12/40 [02:15<03:18,  7.09s/it]

...

100%|██████████| 40/40 [07:39<00:00,  7.59s/it]

{'eval_loss': 1.6000033617019653, 'eval_accuracy': 0.24875621890547264, 'eval_runtime': 21.6838, 'eval_samples_per_second': 9.27, 'eval_steps_per_second': 0.6, 'epoch': 10.0}


100%|██████████| 40/40 [07:47<00:00, 11.70s/it]

{'train_runtime': 468.1127, 'train_samples_per_second': 1.068, 'train_steps_per_second': 0.085, 'train_loss': 1.5118670463562012, 'epoch': 10.0}


TrainOutput(global_step=40, training_loss=1.5118670463562012, metrics={'train_runtime': 468.1127, 'train_samples_per_second': 1.068, 'train_steps_per_second': 0.085, 'train_loss': 1.5118670463562012, 'epoch': 10.0})

In [13]:
tokenizer3 = AutoTokenizer.from_pretrained("MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")
data_collator3 = DataCollatorWithPadding(tokenizer=tokenizer3)
model4 = AutoModelForSequenceClassification.from_pretrained(
    "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli", num_labels=5, id2label = id2label, label2id = label2id, ignore_mismatched_sizes=True
)

def preprocess_function(examples):
    return tokenizer3(examples["text"], truncation=True)

train2 = train.map(preprocess_function)
test2 = test.map(preprocess_function)

trainer3 = Trainer(
    model=model4,
    args=training_args,
    train_dataset=train2,
    eval_dataset=test2,
    tokenizer=tokenizer3,
    data_collator=data_collator3,
    compute_metrics=compute_metrics,
)

trainer3.train()

c:\Python311\Lib\site-packages\transformers\convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able t

MlflowException: Changing param values is not allowed. Param with key='torch_dtype' was already logged with value='None' for run ID='4187cd981e3d49f0b589f5f4663d93b9'. Attempted logging new value 'float16'.